SVM example: Identifyng Chronic Kidney Disease
----

# Introduction

This notebook walks through an example implementing a Support Vector Machine to tackle a medical classification problem. The data comes from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease), based on research by P. Soundarapandian, L. Jerlin Rubini, and P. Esweran.


## Outline

1. Implementing a linear SVM
1. Applying and tuning the model
1. Implementing and applying kernel SVM

## Setup

In [1]:
import numpy as np
import pandas as pd

# 1. Implementing a Linear SVM

## Theory

Recall that a **Support Vector Machine** seeks to minimize classification errors while maximizing the width of the margin around the decision boundary, in hopes of increasing generalizability. More formally, we have a dataset of $n$ feature vectors $x^{(i)}$ and labels $y^{(i)}\in {-1,1}$. We want to find a set of parameters ($\theta, \theta_0$) that mimimize the cost function:
$$ J(\theta , \theta _0) = \frac{1}{n} \sum _{i=1}^{n} \text {Loss}_ h (y^{(i)} (\theta \cdot x^{(i)} + \theta _0 )) + \frac{\lambda }{2} \mid \mid \theta \mid \mid ^2$$
where
$$\text{Loss}_h(z)=\begin{cases}0 & z \ge 1 \\1-z & z < 1\end{cases}$$
and $\lambda$ is the **regularization parameter**, determining the relative weight given to margin width (i.e. the inverse of the squared norm of $\theta$) and accuracy (i.e. the `Loss` term). Higher $\lambda$ focuses more on regularization by widening the margin.

We estimate the parameters through **Stochastic Gradient Descent (SGD)**. This means we randomly select an observation $i$ and update the parameters as follows:

$$\theta \leftarrow \theta - \eta \nabla _{\theta } \big [\text {Loss}_ h(y^{(i)}(\theta \cdot x^{(i)} + \theta _0) ) + \frac{\lambda }{2}\mid \mid \theta \mid \mid ^2 \big ]$$
where $\nabla$ is the learning rate. This can be a constant or can be adjusted over the training process.

If the case is placed correctly and outside the classification margin by the current parameters (i.e. $y^{(i)}(\theta \cdot x^{(i)} + \theta _0)\ge 1$), the update is based strictly on the regularization term. So, the gradient is $\lambda \theta$.

If there is a positive `Loss` (i.e. $y^{(i)}(\theta \cdot x^{(i)} + \theta _0)<1$), then the gradient is $-y^{(i)}x^{(i)} + \lambda \theta$.

We continue cycling through the data, shuffling after each epoch, until the change in the cost remains below some threshold.

Note that this algorithm only optimizes the parameters $\theta$, and $\theta_0$, while $\lambda$, $\nabla$, and the convergence threshold are **hyperparameters** that must be set in advance.

NOTE ABOUT THE OFFSET: rather than treating $\theta_0$ as a separate parameter, it is possibe instead to prepend a `1` to every feature vector, so that the SVM trains $\theta$ as a $D+1$ vector, where the first entry is equivalent to $\theta_0$. This is what will be done below.

## Implementation

The cell below implements a linear SVM as a class that mimics the syntax of the popular machine learning package [scikit-learn](https://scikit-learn.org/stable/index.html).
1. The model is first initialized with given (default or custom) hyperparameters. Here those are the regularization paramter `lam` (for $\lambda$), the learning rate `lrate`, and the convergence `threshold`.
1. Then, the model is trained on data using the method `.fit(X, y)`, where X contains the features vectors (observations as rows, features as columns) and y contains the labels. The model saves the trained parameters.
1. After that, the model can be used to assign labels to any feature vectors with the method `.predict(X)`

The class below also contains three helper functions called by `.predict()`:
1. `.cost()` calculates the cost function for the entire dataset, needed to check for convergence
1. `.loss()` calculates and returns the Loss function for calculating the cost function, again as defined above
1. `.train_epoch()` runs through the observations in random order one time, updating the parameters accordingly, and returning the resulting cost function.

In [2]:
class LinearSVM():
    def __init__(self, lam=1, lrate=0.9, threshold=1e-4) -> None:
        '''
        Initialize the model with the given hyperparameters:
        - `lam`: the regularization parameter lambda
        - `lrate`: the learning rate
        - `threshold`: the convergence threshold
        '''
        self.lam = lam
        self.lrate = lrate
        self.threshold = threshold
        self.theta = None

    def loss(self, Xi, yi):
        '''The Loss function for one observation'''
        if self.theta is None:
            raise ValueError('Cannot calculate loss without initializing theta')

        agreement = yi*(self.theta @ Xi)
        return 0 if agreement >= 1 else 1-agreement
    
    def cost(self, X, y):
        '''Average cost for the whole dataset'''
        loss_sum = 0
        for Xi, yi in zip(X, y):
            loss_sum += self.loss(Xi, yi)
        return loss_sum/len(y) + self.lam/2 * (self.theta@self.theta)
    
    def train_epoch(self, X, y):
        '''Cycles through cases in random order, updating theta. Returns the average cost after every update'''
        rng = np.random.default_rng()
        order = rng.shuffle(np.arange(len(y)))
        for i in order:
            grad = self.lam * self.theta
            grad -= y[i]*X[i] if self.loss(X[i], y[i]) > 0 else 0
            self.theta -= self.lrate * grad
        return self.cost(X, y)
    
    def fit(self, X, y):
        '''Fits the model to the data (feature vectors X, labels y). To fit with an offset, prepend `1` to every feature vector'''
        # initialize theta
        self.theta = np.zeros(X.shape[-1])

        last_cost = np.inf
        new_cost = 0
        while last_cost - new_cost > self.threshold:
            new_cost = self.train_epoch(X, y)
        return self
    
    def predict(self, X):
        '''Predicts labels for feature vectors X'''
        return sign(X @ self.theta)

@np.vectorize
def sign(z):
    '''Converts values to labels, i.e. 1 and -1'''
    return 1 if z>=0 else -1

# 2. Applying and Tuning the Model